In [1]:
import pandas as pd
import numpy as np
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g3
g3=rdflib.Graph()
g3.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_IndependentDept_3.nt',format='nt')
print(len(g3))

11807


In [3]:
#************Change Path to files************
ministry_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/ministry_new_working.csv", header=None) 
ministry_data=ministry_data.replace(np.nan,"",regex=True)

In [4]:
ministry_data.columns=['Cat1','Cat2','Org1','Cat3','Cat4_Org2','Cat5_Org3','Org4']

In [5]:
#Function:Changing plural to singular
def change_cat(col_nam,df):
    singular_plural_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Singular_Plural.csv") 
    for i in range(singular_plural_data.shape[0]):
        val=singular_plural_data.iloc[i,0]
        new_val=singular_plural_data.iloc[i,1]
        df.loc[(df[col_nam]).apply(lambda x: x.strip())==val,col_nam]=new_val
    return(df)

In [6]:
#Function Call:Changing plural to singular
ministry_data=change_cat('Cat1',ministry_data)
ministry_data=change_cat('Cat2',ministry_data)
ministry_data=change_cat('Cat3',ministry_data)
ministry_data=change_cat('Cat4_Org2',ministry_data)
ministry_data=change_cat('Cat5_Org3',ministry_data)

In [7]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [8]:
pd.set_option('display.max_colwidth', 500)#housekeeping

In [9]:
#finding n_org max
global n_org
org_list=[]
for s,p,o in g3.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    org_list.append(num_i) #list of n_org identifier values
org_list.sort() #sorting the list
n_org=org_list[-1] #the current maximum value of n_org

#No need to find n_cat max, just one skos:category- topGOT
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g3.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [10]:
print(n_org)
print(n_cat)

1477
15


In [11]:
# #Clean org1/org2 and return list
# def org_scrub(org_n):
#     org_str_raw=org_n
#     org_str_raw=org_str_raw.replace('"','\'')
#     org_str_raw=org_str_raw.replace("' '","','")
#     org_str_raw=org_str_raw.replace("[","")
#     org_str_raw=org_str_raw.replace("]","")
#     org_str_raw=org_str_raw.replace("'","")
#     org_str_raw=org_str_raw.strip()
#     l=org_str_raw.split(',')
#     l_scrub=[] #Cleaned list of 
#     if len(l)>0:
#         for x in l:
#             x.strip()
#             if x=='':
#                 continue
#             else:
#                 x=x.replace("'","")
#                 l_scrub.append(x)
#     return(l_scrub)


In [12]:
def org_scrub(org_n):
    l_scrub=[]
    val=org_n
    val=str(val)
    val=val.replace('[',"")
    val=val.replace(']',"")
    if len(val)==0:
        return(l_scrub)
    #print(val)
    pos_url=val.find('http')
    #print(pos_url)
    if pos_url!=-1:
        val_1=val[:pos_url]+'|'+val[pos_url:]
        pos_end_url=val_1[pos_url+1:].find("'")
        val_2=val_1[:pos_url+1+pos_end_url]+'|'+val_1[pos_url+1+pos_end_url:]
    else:
        sch_ch=val[0]
        pos_end=val[1:].find(sch_ch)
        #print(val[:pos_end+2])
        val_2=val[:pos_end+2]+"|"+val[pos_end+2:]
        #print(val_2)
    org_ls=val_2.split('|')
    
    for i in range(len(org_ls)):
        ele=org_ls[i]
        ele_ls=list(ele)
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[0] in ["'",'"',',',' '] and len(ele_ls)>0:
                ele_ls.pop(0)
                #print(ele_ls)
            if len(ele_ls)>0:
                if ele_ls[0] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[-1] in ["'",'"',',',' ']:
                ele_ls.pop(-1)
            if len(ele_ls)>0:
                if ele_ls[-1] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        ele_j=''.join(ele_ls)
        l_scrub.append(ele_j)
    return(l_scrub)

In [13]:
#create fetch function for cat_id
def fetch_n_cat(cat):
    global n_cat
    global g3
    cat_id=""
    for s,p,o in g3.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        #adding relevant triples
#         g3.add((cat_id,skos.broader, URIRef(bhcatres+'topOT')))
#         g3.add((URIRef(bhcatres+'topOT'),skos.broaderOf,cat_id))
        g3.add((cat_id,RDF.type,skos.Concept))
        g3.add((cat_id,skos.prefLabel, Literal(cat)))
        g3.add((cat_id,skos.altLabel, Literal(cat)))
        g3.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

#create fetch function for org_id
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g3
    global n_org
    org_id=""
    for s,p,o in g3.triples((None, bhorg.hasName,Literal(org_nam))):
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g3.add((org_id, bhorg.hasName, Literal(org_nam)))
        g3.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        
        if org_url!="":
            g3.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g3.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g3.add((org_id,RDF.type,bhorg.Organisation))
        g3.add((org_id,org.identifier,Literal(n_org)))
        g3.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [14]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g3
    if not (cat2_id, skos.broader,cat1_id) in g3:
        g3.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g3:
        g3.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g3
    if not (org1_id,org.hasSubOrganization,org2_id) in g3:
        g3.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g3:
        g3.add((org2_id,org.subOrganizationOf,org1_id))

In [15]:
#Iteration over ministry_data rows
for i in range(ministry_data.shape[0]):
#     print(i)
    #['Cat1','Cat2','Org1','Cat3','Cat4_Org2','Cat5_Org3','Org4']
    cat1=ministry_data.iloc[i,0].strip()
    cat2=ministry_data.iloc[i,1].strip()
    org1=ministry_data.iloc[i,2]
    cat3=ministry_data.iloc[i,3].strip() #may be blank
    cat4_org2=ministry_data.iloc[i,4]#may be blank
    cat5_org3=ministry_data.iloc[i,5]
    org4=ministry_data.iloc[i,6]
    
    cat1_id=""
    cat2_id=""
    cat3_id=""
    cat4_id=""
    cat5_id=""
    org1_id=""
    org2_id=""
    org3_id=""
    org4_id=""
    
    #check if cat1_id already exists, else create
    cat1_id=fetch_n_cat(cat1)
    
    #check if ((cat1 broader cat21)), ((cat21 broaderOf cat1)) exist, else create
    cat_govt=URIRef(bhcatres+'21')
    fetch_cat_broader(cat_govt,cat1_id)

    #check if cat2_id already exists, else create
    cat2_id=fetch_n_cat(cat2)
    
    #check if ((cat2 broader cat1)), ((cat1 broaderOf cat2)) exist, else create
    fetch_cat_broader(cat1_id,cat2_id)#*****************CHECK/CHANGE THIS*****************#
        
    #check if cat3_id already exists, else create
    ##processing cat3- removing'[]'
    cat3=cat3.replace('[',"")
    cat3=cat3.replace(']',"")
    if cat3=="":
        cat3_id=cat2_id
    else:
        cat3_id=fetch_n_cat(cat3)
        #check if ((cat2 broader cat3)), ((cat2 broaderOf cat3)) exist, else create
        fetch_cat_broader(cat2_id,cat3_id)#*****************CHECK/CHANGE THIS*****************#
    
    #check if org1_id already exists, else create
    ##clean and return list
#     org1_ls=org_scrub(org1)
#     org_nam=""
#     org_url=""
#     org_abb=""
#     print(org1_ls)
#     if len(org1_ls)==1:
#         org_nam=org1_ls[0]
# #     elif len(org1_ls)==2:
# #         org_nam=org1_ls[0]
# #         org_url=org1_ls[1]
#     elif len(org1_ls)==2:
#         org_nam=org1_ls[0]
#         if org1_ls[1].find('http')>-1:
#             org_url=org1_ls[1]
#         else:
#             org_abb=org1_ls[1]
#     else:
#         org_nam=org1_ls[0]
#         org_url=org1_ls[1]
#         org_abb=org1_ls[2]
#     print(org_nam," ",org_url," ",org_url)
    
    org1=org1.replace('[',"")
    org1=org1.replace(']',"")
    org1=org1.replace('\'',"")
        
    org_nam=org1
    
    org1_id=fetch_n_org(org_nam)
    
    #create tuple org1, org:classification, cat2_id
    g3.add((org1_id,org.classification,cat2_id)) 
    
    #create tuples for cat4_org2
    if cat4_org2=="":
        #cat4_id=""
        cat4_id=cat3_id #only needed for broader relation- check if can be removed
        #org2_id=""
        org2_id=org1_id #only needed for sub-org relation- check if can be removed   
    elif cat4_org2[0]=='[':
        #create org2 tuples
        org2_ls=org_scrub(cat4_org2)
        org_nam=""
        org_url=""
        org_abb=""
        if (len(org2_ls))>0: 
            if len(org2_ls)==1:
                org_nam=org2_ls[0]
#             elif len(org2_ls)==2:
#                 org_nam=org2_ls[0]
#                 org_url=org2_ls[1]
            elif len(org2_ls)==2:
                org_nam=org2_ls[0]
                if org2_ls[1].find('http')>0:
                    org_url=org2_ls[1]
                else:
                    org_abb=org2_ls[1]
            else:
                org_nam=org2_ls[0]
                org_url=org2_ls[1]
                org_abb=org2_ls[2]
            org2_id=fetch_n_org(org_nam,org_url,org_abb)
        #create tuple org2, org:classificatoin, cat2_id
            g3.add((org2_id,org.classification,cat3_id))
        #create tuples for org1-->subOrg-->org2
            fetch_org_subOrg(org1_id,org2_id)
        #As cat4_id=""
        cat4_id=cat3_id

    else:
        #create cat4 tuples
        cat4_org2=cat4_org2.strip()
        cat4_id=fetch_n_cat(cat4_org2)
        #check if ((cat4 broader cat3)), ((cat3 broaderOf cat4)) exist, else create
        fetch_cat_broader(cat3_id,cat4_id)
        org2_id=org1_id
    #cat5_org3
    #create tuples for cat5_org3
    if cat5_org3=="":
        #chekc if really needed**************************
        cat5_id=cat4_id
        org3_id=org2_id
    elif cat5_org3[0]=='[':
        #create org2 tuples
        org3_ls=org_scrub(cat5_org3)
        org_nam=""
        org_url=""
        org_abb=""
        
        if (len(org3_ls))>0: 
            if len(org3_ls)==1:
                org_nam=org3_ls[0]
#             elif len(org3_ls)==2:
#                 org_nam=org3_ls[0]
#                 org_url=org3_ls[1]
            elif len(org3_ls)==2:
                org_nam=org3_ls[0]
                if org3_ls[1].find('http')>0:
                    org_url=org3_ls[1]
                else:
                    org_abb=org3_ls[1]
            else:
                org_nam=org3_ls[0]
                org_url=org3_ls[1]
                org_abb=org3_ls[2]
            org3_id=fetch_n_org(org_nam,org_url,org_abb)
            #create tuple org2, org:classificatoin, cat2_id
            g3.add((org3_id,org.classification,cat4_id))
            #create tuples for org2-->subOrg-->org3
            fetch_org_subOrg(org2_id,org3_id)
        cat5_id=cat4_id
    else:
        #create cat5 tuples
        cat5_org3=cat5_org3.strip()
        cat5_id=fetch_n_cat(cat5_org3)
        #check if ((cat5 broader cat4)), ((cat4 broaderOf cat5)) exist, else create
        fetch_cat_broader(cat4_id,cat5_id)
        org3_id=org2_id
    #Create tuples for org_4    
    org4=org4.replace('[',"")
    org4=org4.replace(']',"")
    
    if org4!="":
        org4_ls=org_scrub(org4)
        org_nam=""
        org_url=""
        org_abb=""
        if len(org4_ls)==1:
            org_nam=org4_ls[0]
#         elif len(org4_ls)==2:
#             org_nam=org4_ls[0]
#             org_url=org4_ls[1]
        elif len(org4_ls)==2:
            org_nam=org4_ls[0]
            if org4_ls[1].find('http')>0:
                org_url=org4_ls[1]
            else:
                org_abb=org4_ls[1]
        else:
            org_nam=org4_ls[0]
            org_url=org4_ls[1]
            org_abb=org4_ls[2]
        org4_id=fetch_n_org(org_nam,org_url,org_abb)
        #create tuple org4, org:classificatoin, cat5_id
        g3.add((org4_id,org.classification,cat5_id))
        #create tuples for org3-->subOrg-->org4
        fetch_org_subOrg(org3_id,org4_id)
        

In [16]:
print(len(g3))

38871


In [17]:
#print(i)

In [18]:
g3.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_ministry_4.nt')

In [19]:
#Testing
for s,p,o in g3.triples((None, bhorg.hasName,Literal("SMDPII - VLSI: Special Manpower Development Programme in the area of VLSI Design"))):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/org/12265 http://semanticwebindia.in/bharathi/ont/org/hasName SMDPII - VLSI: Special Manpower Development Programme in the area of VLSI Design


In [20]:
u=URIRef("http://semanticwebindia.in/bharathi/res/org/11477")
for s,p,o in g3.triples((u,None,None)):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://semanticwebindia.in/bharathi/ont/org/Organisation
http://semanticwebindia.in/bharathi/res/org/11477 http://semanticwebindia.in/bharathi/ont/org/hasAbbreviation ISAC
http://semanticwebindia.in/bharathi/res/org/11477 http://semanticwebindia.in/bharathi/ont/org/hasName ISRO Satellite Centre(ISAC) Bengaluru
http://semanticwebindia.in/bharathi/res/org/11477 http://semanticwebindia.in/bharathi/ont/org/hasURL http://www.isac.gov.in
http://semanticwebindia.in/bharathi/res/org/11477 http://semanticwebindia.in/bharathi/ont/org/hasAlternateName ISRO Satellite Centre(ISAC) Bengaluru
http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/ns/org#subOrganizationOf http://semanticwebindia.in/bharathi/res/org/11461
http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/213
http://semanticwebindia.in/bharathi/

In [21]:
for s,p,o in g3.triples((None,None,Literal("Ministry of Civil Aviation"))):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/org/121 http://semanticwebindia.in/bharathi/ont/org/hasAlternateName Ministry of Civil Aviation
http://semanticwebindia.in/bharathi/res/org/121 http://semanticwebindia.in/bharathi/ont/org/hasName Ministry of Civil Aviation


In [22]:
val=(ministry_data.iloc[306,2]).replace('[',"")
val=val.replace(']',"")

In [23]:
print(val)

'Ministry of Coal'
